In [1]:
import requests, re

from bs4 import BeautifulSoup
from googlesearch import search
from GoogleNews import GoogleNews
from googletrans import Translator

from joblib import Parallel, delayed
from tqdm import tqdm



In [2]:

def clean_spaces_text(text):
    # Replace multiple spaces, tabs, and newline characters with a single space
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()  # Remove leading and trailing spaces

def search_google(query, number_of_results):
    urls = []
    for url in search(query, num_results=number_of_results):
        urls.append(url)
    return urls


def fetch_articles(url_to_search): # FETCH THE CONTENT FROM THE WEBPAGE
    try:
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

        response = requests.get(url_to_search, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract text from webpage
            text_content = ' '.join([p.text for p in soup.find_all('p')])
            return text_content
        else:
            print("Failed to fetch webpage:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", str(e))
        return None
    
    
def search_google_news(query,                   # QUERY / KEYWORDS TO SEARCH
                       number_of_results = 5,   # NUMBER OF ARTICLES / NEWS TO FETCH
                       period = "7d"):          # PAST DAYS
    news_urls = []
    googlenews = GoogleNews()
    
    # FILTERS AND PARAMETERS
    googlenews.set_lang("en")
    googlenews.set_period(period)
    googlenews.set_encode("utf-8")

    googlenews.search(query)
    results = googlenews.results()
    print(results)
    for i in range(min(len(results), number_of_results)):
        news_urls.append(results[i]['link'])
    googlenews.clear()
    return news_urls

def contains_hindi(text): # CHECK IF THE TEXT CONTAINS HINDI WORDS/CHARACTERS
    hindi_pattern = re.compile(r'[\u0900-\u097F]+')
    if hindi_pattern.search(text):
        print("contains hindi !")
        return True
    return False

def clean_links(link):
    link = link.split("&ved")[0] # IDK WHAT THIS "&ved" IS ... IT GOT ATTATCHED AUTOMATICALLY... DUE TO WHICH IT WAS GIVING ERROR 404 
    return link

def translate_hindi_to_english(searched_data_dict):
    translator = Translator()
    for key, value in searched_data_dict.items():
        if contains_hindi(value):
            print(f"{key} - Contains Hindi")
            try:
                translated_text = translator.translate(value, dest="en")
                searched_data_dict[key] = translated_text.text
            except Exception as e:
                print("Error translating. Error:", e)


In [3]:
query = "Sun Earth"
number_of_results = 5


In [4]:
# Search Google for the query
search_results = search_google(query, number_of_results)

# Fetch articles and extract text content
google_search_data = {}
for url in search_results:
    article_text = fetch_articles(url)
    if article_text:
        google_search_data[url] = clean_spaces_text(article_text)
        print("URL:", url)
        print("Text content:", clean_spaces_text(article_text))
        print()


URL: https://science.nasa.gov/sun/
Text content: The Sun is the star at the heart of our solar system. Its gravity holds the solar system together, keeping everything – from the biggest planets to the smallest bits of debris – in its orbit. The Sun's gravity holds the solar system together, keeping everything – from the biggest planets to the smallest particles of debris – in its orbit. The connection and interactions between the Sun and Earth drive the seasons, ocean currents, weather, climate, radiation belts and auroras. Though it is special to us, there are billions of stars like our Sun scattered across the Milky Way galaxy. The Sun has many names in many cultures. The Latin word for Sun is “sol,” which is the main adjective for all things Sun-related: solar. The Sun has inspired us since ancient times. It’s central to mythology and religion in cultures around the world, including the ancient Egyptians, the Aztecs of Mexico, Native American tribes of North and South America, the C

In [5]:
# query = "Election in india"
# number_of_results = 5

# Search Google News for the query
news_links = search_google_news(query, number_of_results)
news_links = [clean_links(i) for i in news_links]
print(news_links)


[{'title': 'Conspiracy Theorists Are Now Claiming The Earth Has Two Suns', 'media': 'BroBible', 'date': '6 hours ago', 'datetime': datetime.datetime(2024, 5, 2, 19, 41, 15, 596805), 'desc': "The latest wild conspiracy theory making the rounds on the internet is that there is not just one, but two suns in Earth's solar system.", 'link': 'https://brobible.com/culture/article/conspiracy-theorists-earth-has-two-suns/&ved=2ahUKEwjY6Yuj4--FAxVX3TgGHVwdBaEQxfQBegQIAhAC&usg=AOvVaw14nJA82jF1PUSFknWRsjws', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': 'Radio communication disappeared on Earth due to a solar flare', 'media': 'The Universe Space Tech', 'date': '1 hours ago', 'datetime': datetime.datetime(2024, 5, 3, 0, 41, 15, 600501), 'desc': 'On May 1, an extremely powerful solar flare occurred on the Sun, which led to a large-scale radio blackout throughout the Pacific region.', 'link': 'https://universemagazine.com/en/powerful-solar-fla

In [6]:
google_news_data = {}

# Fetch articles and extract text content
for url in news_links:
    article_text = fetch_articles(url)
    if article_text:
        google_news_data[url] = clean_spaces_text(article_text) 
        print("URL:", url)
        print("Text content:", clean_spaces_text(article_text))
        print()


URL: https://brobible.com/culture/article/conspiracy-theorists-earth-has-two-suns/
Text content: iStockphoto If you enjoy a good conspiracy theory then today is your lucky day as there are some people out there trying to convince others that the Earth has two suns. That’s right. It appears that this latest conspiracy theory has its origin, or at least has resurfaced, thanks to a Facebook group called Nibiru Followers Anonymous. If the term Nibiru sounds familiar that is because it also goes by the name Planet X (not to be confused with Earth X or the other Planet X) and it is supposedly going to slam into Earth at some point ending life as we know it. According to this conspiracy theory about the Earth having two suns, the reason we haven’t noticed that there are two suns is because one of them is hiding behind the other one. This is something they believe. Their proof of there being a second sun is that on occasion when looking at the sun there appears to be a ring or halo of light ar

In [8]:
translator = Translator()
for i in google_news_data:
    if contains_hindi(google_news_data[i]):
        print("Contains hindi")
        try :
            google_news_data[i] = translator.translate(google_news_data[i], dest="en")
        except Exception as e:
            print("Error translating. Error :", e)
            


In [9]:
translator = Translator()
for i in google_search_data:
    if contains_hindi(google_search_data[i]):
        print("Contains hindi")
        try :
            google_search_data[i] = translator.translate(google_search_data[i], dest="en")
        except Exception as e:
            print("Error translating. Error :", e)
            


contains hindi !
Contains hindi


In [18]:
testing_dict = {"xyz": "हैलो, कैसे हो भाई ?. all good ?"}

In [7]:
translate_hindi_to_english(google_news_data)
translate_hindi_to_english(google_search_data)

contains hindi !
https://en.wikipedia.org/wiki/Sun - Contains Hindi


In [16]:
testing_dict

{'xyz': 'Hello, how are you brother? all good?'}

In [8]:
len(google_news_data), len(google_search_data)

(5, 7)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForQuestionAnswering, AutoModelForSeq2SeqLM
from transformers import pipeline, set_seed
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")

In [5]:

translate_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
translate_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large").to(device)

# translate_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)

In [15]:
def translate_eng(text, verbose = False):
    prompt = f"Translate this text to English language : {text} "
    print("Given text :", text)
    input_ids = translate_tokenizer(prompt,
                                    return_tensors="pt",
                                    # padding=True,
                                    # truncation=True,
                                    ).to(translate_model.device).input_ids
    outputs = translate_model.generate(input_ids, 
                                       max_new_tokens = 512)
    model_translation = translate_tokenizer.decode(outputs[0])
    return model_translation

In [5]:
good_text = clean_spaces_text(bad_text)
good_text

'मेरा शहर Link Copied रहें हर खबर से अपडेट, डाउनलोड करें Android Hindi News App, iOS Hindi News App और Amarujala Hindi News APP अपने मोबाइल पे|Get all India News in Hindi related to live update of politics, sports, entertainment, technology and education etc. Stay updated with us for all breaking news from India News and more news in Hindi. Next Article Please wait... Please wait... Delete All Cookies Followed news.google.com/articles/CBMiiQFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhhLWVsZWN0aW9uLTIwMjQtc2Vjb25kLXBoYXNlLXZvdGluZy1pcy1zaW1pbGFyLXRvLXRoZS1maXJzdC1waGFzZS13aGF0LWlzLXRoZS1tZWFuaW5nLW9mLXRoaXMtdHJlbmQtNTUzMDc0M9IBjwFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhhLWVsZWN0aW9uLTIwMjQtc2Vjb25kLXBoYXNlLXZvdGluZy1pcy1zaW1pbGFyLXRvLXRoZS1maXJzdC1waGFzZS13aGF0LWlzLXRoZS1tZWFuaW5nLW9mLXRoaXMtdHJlbmQtNTUzMDc0My9hbXAvMQ?hl=hi&gl=IN&ceid=IN%3Ahi Articles : लोकसभा चुनाव 2024 (Lok Sabha Election 2024) के दूसरे चरण में शुक्रवार को 13 राज्यों और केंद्र शासित प्रदेशों की 88 सीटों पर मतदान हुए. हाल

In [6]:
# SPLITTING LARGE DOCS IN SMALLER CHUNKS
text_splitter = RecursiveCharacterTextSplitter(chunk_size=156, chunk_overlap=50)
chunks = text_splitter.split_text(good_text)
chunks

['मेरा शहर Link Copied रहें हर खबर से अपडेट, डाउनलोड करें Android Hindi News App, iOS Hindi News App और Amarujala Hindi News APP अपने मोबाइल पे|Get all India',
 'Hindi News APP अपने मोबाइल पे|Get all India News in Hindi related to live update of politics, sports, entertainment, technology and education etc. Stay',
 'entertainment, technology and education etc. Stay updated with us for all breaking news from India News and more news in Hindi. Next Article Please wait...',
 'more news in Hindi. Next Article Please wait... Please wait... Delete All Cookies Followed',
 'news.google.com/articles/CBMiiQFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhhLWVsZWN0aW9uLTIwMjQtc2Vjb25kLXBoYXNlLXZvdGluZy1pcy1zaW1pbGFyLXRvLXRoZS1maXJzdC1waG',
 'YXNlLXZvdGluZy1pcy1zaW1pbGFyLXRvLXRoZS1maXJzdC1waGFzZS13aGF0LWlzLXRoZS1tZWFuaW5nLW9mLXRoaXMtdHJlbmQtNTUzMDc0M9IBjwFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhh',
 'c0M9IBjwFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhhLWVsZWN0aW9uLTIwMjQtc2Vjb25kLXBoYXNlLXZvdGluZy1pcy1zaW1p

In [43]:
gnews.clear()

In [20]:
translate_eng(chunks[10])

Given text : इस बार का वोटिंग ट्रेंड पहले चरण के चुनाव से भी खराब रहा. दूसरे चरण में महज 63.00 प्रतिशत मतदाताओं ने ही अपने मताधिकार का प्रयोग किया, जबकि 2019 में इन्हीं


'<pad> The 63.00 kilometer journey from the airport to the city centre takes about 63 hours, and the return journey takes about 63 hours.</s>'

In [1]:
from llama3 import llama3
from nltk import sent_tokenize


In [7]:
# SPLITTING LARGE DOCS IN SMALLER CHUNKS
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunks = text_splitter.split_text(good_text)
chunks

['मेरा शहर Link Copied रहें हर खबर से अपडेट, डाउनलोड करें Android Hindi News App, iOS Hindi News App और Amarujala Hindi News APP अपने मोबाइल पे|Get all India News in Hindi related to live update of politics, sports, entertainment, technology and education',
 'sports, entertainment, technology and education etc. Stay updated with us for all breaking news from India News and more news in Hindi. Next Article Please wait... Please wait... Delete All Cookies Followed',
 'news.google.com/articles/CBMiiQFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhhLWVsZWN0aW9uLTIwMjQtc2Vjb25kLXBoYXNlLXZvdGluZy1pcy1zaW1pbGFyLXRvLXRoZS1maXJzdC1waGFzZS13aGF0LWlzLXRoZS1tZWFuaW5nLW9mLXRoaXMtdHJlbmQtNTUzMDc0M9IBjwFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLX',
 'NTUzMDc0M9IBjwFodHRwczovL25kdHYuaW4vaW5kaWEvbG9rLXNhYmhhLWVsZWN0aW9uLTIwMjQtc2Vjb25kLXBoYXNlLXZvdGluZy1pcy1zaW1pbGFyLXRvLXRoZS1maXJzdC1waGFzZS13aGF0LWlzLXRoZS1tZWFuaW5nLW9mLXRoaXMtdHJlbmQtNTUzMDc0My9hbXAvMQ?hl=hi&gl=IN&ceid=IN%3Ahi',
 'Articles : लोकसभा चुनाव 2024 (Lok

In [2]:
llama3_rag_model = llama3()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
translated_chunks = []
for chunk in tqdm(chunks):
    translated_chunks.append(llama3_rag_model.llama3_eng_translator(good_text, verbose=1))

  0%|          | 0/20 [00:00<?, ?it/s]

e:\Downloads\WeLearn\9th Sem\nineSemEnv\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
e:\Downloads\WeLearn\9th Sem\nineSemEnv\Lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
e:\Downloads\WeLearn\9th Sem\nineSemEnv\Lib\site-packages\transformers\generation\configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were no


 --- Generating outputs --- 




  0%|          | 0/20 [04:18<?, ?it/s]


KeyboardInterrupt: 

In [3]:
import torch, gc

In [11]:
gc.collect()

11987

In [12]:
del llama3_rag_model

NameError: name 'llama3_rag_model' is not defined

In [13]:
torch.cuda.empty_cache()

In [3]:
# from google.transliteration import transliterate_text

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:
def contains_hindi(text):
    hindi_pattern = re.compile(r'[\u0900-\u097F]+')
    if hindi_pattern.search(text):
        print("contains hindi !")
        return True
    return False

In [12]:
import googletrans

In [10]:
translator = Translator()

In [11]:
eng_good_text = ""
c = chunks[6]
if contains_hindi(c):
    print("Given Text =", c)
    eng_good_text = translator.translate(c, src="hi", dest="en").text

eng_good_text

contains hindi !
Given Text = बढ़-चढ़कर वोट किया था. कम होते इस वोटिंग प्रतिशत ने सभी राजनीतिक दलों का गणित बिगाड़ दिया है. पहले चरण में 21 राज्यों की 102 लोकसभा सीटों पर 64 प्रतिशत वोट डाले गए थे. पिछले चुनाव में उन सीटों पर भी 70 प्रतिशत से ज्यादा मतदान हुए थे. यही हाल दूसरे चरण में


'Voted enthusiastically. This decreasing voting percentage has spoiled the mathematics of all political parties. In the first phase, 64 percent votes were cast on 102 Lok Sabha seats in 21 states. In the last elections, more than 70 percent voting took place on those seats also. Same situation in the second phase'

In [13]:
# import googletrans
# print(googletrans.__version__)

3.1.0-alpha
